In [1]:
#set up autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
from pathlib import Path

import importlib  
fa_etl = importlib.import_module("fa-etl")

initial data quality checks

1. Check if all parcels in annual file exist in tax & val hist:
    * 36061: yes
    * 42101: not all (606/582083 of props in annual are not in tax hist, all in value hist)
    * 48203: yes

In [3]:
county = "42101"
path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"

In [7]:
taxhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ValHist{county}.parquet")
annual = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
sales = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")

annual = annual.with_columns([
    pl.col("PropertyID").is_in(taxhist['PropertyID']).alias("in_taxhist"),
    pl.col("PropertyID").is_in(valhist['PropertyID']).alias("in_valhist")
    ])

In [ ]:
annual["in_taxhist"].value_counts()

In [ ]:
annual["in_valhist"].value_counts()

In [ ]:
annual.filter([
    pl.col("in_taxhist") == False
])
    

data quality checks:
* check for composition of valhist
* why am i losing data when i merge valhist with prop? 2871980 to 2868382

In [ ]:
valhist.filter(
    ((pl.col("AssdTotalValue") == 0) & ((pl.col("MarketValueYear").is_not_null()) | (pl.col("ApprYear").is_not_null())))
)

In [5]:
valhist.shape

(6337101, 8)

In [8]:
sales.schema

OrderedDict([('PropertyID', Int64),
             ('SaleAmt', Int64),
             ('RecordingYear', Int64)])

In [ ]:
taxhist.schema

In [9]:
fa_etl.join(input_dir="/Users/claireboyd/internships/mansueto/firstamerican-etl/dev/36061",
                valhist_filename="ValHist36061.parquet",
                prop_filename="Prop36061.parquet",
                ranked_deed_filename="ranked_Deed36061.parquet")

In [10]:
merged = pl.read_parquet("/Users/claireboyd/internships/mansueto/firstamerican-etl/dev/36061/unified/merged.parquet")

In [11]:
merged.shape

(2555855, 25)

In [12]:
merged.head(10)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,Year,Value,AssessmentUsed,SaleAmt
i64,i64,i64,i64,i64,i64,i64,i64,str,date,f64,f64,str,str,str,str,str,str,str,str,str,i64,i64,str,i64
91849011,109355,2018,243012,2018,0,null,null,"""R""",2024-02-07,40.744385,-73.984104,"""39 E 29TH ST A…","""NEW YORK""","""NY""","""10016""","""36061""","""007400""","""2001""","""B""","""36061_7400""",2018,109355,"""Assd""",null
91849025,232401,2018,516447,2018,0,null,null,"""R""",2024-02-07,40.744385,-73.984104,"""39 E 29TH ST A…","""NEW YORK""","""NY""","""10016""","""36061""","""007400""","""2001""","""B""","""36061_7400""",2018,232401,"""Assd""",null
91848291,41919,2018,93153,2018,0,null,null,"""R""",2024-02-07,40.745345,-73.999298,"""312 W 23RD ST …","""NEW YORK""","""NY""","""10011""","""36061""","""009300""","""2001""","""B""","""36061_9300""",2018,41919,"""Assd""",null
91848380,41919,2018,93153,2018,0,null,null,"""R""",2024-02-07,40.745345,-73.999298,"""312 W 23RD ST …","""NEW YORK""","""NY""","""10011""","""36061""","""009300""","""2001""","""B""","""36061_9300""",2018,41919,"""Assd""",null
91848104,108484,2018,241076,2018,0,null,null,"""R""",2024-02-07,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""","""B""","""36061_8200""",2018,108484,"""Assd""",null
91847856,73548,2018,163440,2018,0,null,null,"""R""",2024-02-07,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""","""B""","""36061_8200""",2018,73548,"""Assd""",null
91847850,1093365,2018,3625000,2018,0,null,null,"""R""",2024-02-07,40.739371,-73.989025,"""24 E 21ST ST #…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""","""B""","""36061_5200""",2018,1093365,"""Assd""",null
91847993,254799,2018,566219,2018,0,null,null,"""R""",2024-02-07,40.729882,-74.007313,"""421 HUDSON ST …","""NEW YORK""","""NY""","""10014""","""36061""","""006900""","""2005""","""B""","""36061_6900""",2018,254799,"""Assd""",null
91847796,297186,2018,660413,2018,0,null,null,"""R""",2024-02-07,40.739961,-73.990361,"""141 5TH AVE AP…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""2000""","""B""","""36061_5200""",2018,297186,"""Assd""",null


In [13]:
merged.filter(
    pl.col("SaleAmt").is_not_null()
)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,Year,Value,AssessmentUsed,SaleAmt
i64,i64,i64,i64,i64,i64,i64,i64,str,date,f64,f64,str,str,str,str,str,str,str,str,str,i64,i64,str,i64
91847870,117677,2018,261504,2018,0,null,null,"""R""",2024-02-07,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""","""B""","""36061_8200""",2018,117677,"""Assd""",1580000
91849008,247162,2018,1462000,2018,0,null,null,"""R""",2024-02-07,40.750447,-73.997723,"""362 W 30TH ST""","""NEW YORK""","""NY""","""10001""","""36061""","""009700""","""4000""","""B""","""36061_9700""",2018,247162,"""Assd""",4400000
91848046,71155,2018,158123,2018,0,null,null,"""R""",2024-02-23,40.738643,-73.987954,"""254 PARK AVE S…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""","""B""","""36061_5200""",2018,71155,"""Assd""",712775
91846549,149310,2018,331800,2018,0,null,null,"""R""",2024-02-07,40.729262,-74.004785,"""63 DOWNING ST …","""NEW YORK""","""NY""","""10014""","""36061""","""006700""","""2002""","""B""","""36061_6700""",2018,149310,"""Assd""",2475000
91847537,128574,2018,285721,2018,0,null,null,"""R""",2024-02-07,40.73889,-73.987904,"""260 PARK AVE S…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""","""B""","""36061_5200""",2018,128574,"""Assd""",1625000
91848333,356895,2018,793100,2018,0,null,null,"""R""",2024-02-07,40.726433,-73.99315,"""40 BOND ST APT…","""NEW YORK""","""NY""","""10012""","""36061""","""005502""","""1002""","""B""","""36061_5502""",2018,356895,"""Assd""",7400000
91846649,219589,2018,487975,2018,0,null,null,"""R""",2024-02-07,40.7408,-73.9989,"""22-26 W 15TH S…","""NEW YORK""","""NY""","""10011""","""36061""","""008100""","""2000""","""5""","""36061_8100""",2018,219589,"""Assd""",3100000
91847215,299300,2018,665110,2018,0,null,null,"""R""",2024-02-23,40.7408,-73.9989,"""12-14 W 18TH S…","""NEW YORK""","""NY""","""10011""","""36061""","""008100""","""2000""","""5""","""36061_8100""",2018,299300,"""Assd""",2825000
91846134,718067,2022,1595704,2022,0,null,null,"""R""",2024-02-07,40.740314,-73.988185,"""23 E 22ND ST #…","""NEW YORK""","""NY""","""10010""","""36061""","""005600""","""2003""","""B""","""36061_5600""",2022,718067,"""Assd""",12500000


In [15]:
sales.filter(
    pl.col('RecordingYear') > 2017
)

PropertyID,SaleAmt,RecordingYear
i64,i64,i64
116347959,305000,2022
116347964,350000,2018
116347965,40000,2018
116347965,485000,2023
116347957,225000,2021
116347955,250000,2022
116347955,235000,2018
116347970,412500,2022
116347972,330000,2018


In [ ]:
sales.schema

In [ ]:
valhist.shape

# property file - propertyid is not null, data type issues

In [ ]:
ranked_sales = pl.read_parquet("/Users/claireboyd/internships/mansueto/firstamerican-etl/dev/36061/staging/ranked_Deed36061.parquet")

In [ ]:
ranked_sales.schema

In [ ]:
valhist.schema